In [3]:
import os
from dotenv import load_dotenv
from IPython.display import display, Image

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.tools.retriever import create_retriever_tool
from langgraph.checkpoint.memory import InMemorySaver

from typing import List
import logging
from langchain_community.tools import BraveSearch
from langchain_tavily import TavilyExtract
import json

# For QA-Retriever
from langchain_core.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Custom Import
from optcg.vectorstore_logic import check_for_updates_to_rules, create_or_load_vectorstore_optcg_rulebooks
from optcg.tools import web_search_tool, youtube_search_tool, create_rulebook_retriever_tool, card_search_tool
from optcg.agents import BaseAgent, RulebookAgent, ToollessAgent_4_1, ToollessAgent_o4_mini, SearchAgent_web_youtube, SearchAgent_web, SearchAgent_youtube, SearchAgent_web_youtube_rulebook, SearchAgent_web_rulebook, SearchAgent_youtube_rulebook


from langsmith import traceable
from langsmith.wrappers import wrap_openai
from openai import OpenAI
#openai_client = wrap_openai(OpenAI())

_ = load_dotenv() # Loads the .env file - e.g. the OPENAI_API_KEY

logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s")


In [2]:
agent = SearchAgent_web_youtube_rulebook()
agent.chat("what is private information?", verbose=True)

2025-07-03 09:37:59,564 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Loading existing vector store...


2025-07-03 09:38:02,847 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-03 09:38:03,548 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-03 09:38:07,416 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content='what is private information?', additional_kwargs={}, response_metadata={}, id='b8f06912-0d7c-43f5-b629-9e3c9a6b9740'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CBMU3Jl5wqowTkZgkyjvrq9C', 'function': {'arguments': '{"query":"private information"}', 'name': 'rulebooks_retriever'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 263, 'total_tokens': 281, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_b3f1157249', 'id': 'chatcmpl-BpFMYNu3p80RcQZrmH6zhf8Y4wrsk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='SearchAgent_web_youtube_rulebook', id='run--e4c1d2c3-f922-4cbc-83a3-72e417d49ed2-0', tool_calls=[{'

In [4]:
class CardDBToolAgent(BaseAgent):
    # If model and temperature are not specified, defaults will be used, defined in BaseAgent.
    def __init__(self, model_name="gpt-4.1"):
        self.name = "CardDBToolAgent"
        super().__init__(model_name=model_name, temperature=0)
    
    def _create_prompt(self):
        """Create a system prompt for the agent. This should be tailored to the specific agent's purpose."""
        return "You are a helpful assistant that helps people answer information about the One Piece TCG. If you don't know the answer, just say you don't know. Do not try to make up an answer."

    def _setup_tools(self):
        """Setup tools for this agent. This can include any tools you want to use, or be an empty list if no tools are needed."""
        return [
            card_search_tool
        ]
agent = CardDBToolAgent(model_name="gpt-4.1")
agent.chat("What is a luffy card with power 2000?", verbose=True)

2025-07-08 10:48:14,168 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-08 10:48:23,571 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content='What is a luffy card with power 2000?', additional_kwargs={}, response_metadata={}, id='f0634f71-2b2f-4773-9a26-2f5360ba79ef'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qql4dwe1156hSAxtHpxKVV0k', 'function': {'arguments': '{"input":{"query":"Luffy","power":2000}}', 'name': 'card_search_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 227, 'total_tokens': 250, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-Br4qDTNS62PBFT25NEzWWTErLsRvw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='CardDBToolAgent', id='run--4c53ad13-a875-43d3-87f3-243e226e6315-0', tool_calls=[{'na

In [3]:
from optcg.api import CardSearchRequest

In [ ]:
from optcg.tools import youtube_search_tool
from dotenv import load_dotenv

_ = load_dotenv()

youtube_search_tool("optcg tourney finals 2024")

[{'url': 'https://www.youtube.com/watch?v=MaWHADL2mCg&pp=ygUZb3B0Y2cgdG91cm5leSBmaW5hbHMgMjAyNA%3D%3D',
  'raw_content': '# Ryuki Okada VS Natalie Millar - Pokémon TCG Masters Finals | EUIC 2025\n## The Official Pokémon YouTube channel\n6490000 subscribers\n1419 likes\n\n### Description\n104875 views\nPosted: 24 Feb 2025\nWatch Ryuki Okada Vs Natalie Millar in the Pokémon TCG Masters Finals at EUIC 2025!   \n\n🏆 Want to learn more about the Pokémon Championship Series? Get in the game over on our website: https://www.pokemon.com/us/play-pokemon   \n    \nOfficial site: https://www.pokemon.com/    \nShop: http://www.pokemoncenter.com    \nFacebook: http://www.facebook.com/Pokemon    \nTwitter: http://www.twitter.com/PlayPokemon    \nInstagram: http://www.instagram.com/playpokemon\n\n235 comments\n### Transcript:',
  'images': []},
 {'url': 'https://www.youtube.com/watch?v=D0n0k_kL6Ro&pp=ygUZb3B0Y2cgdG91cm5leSBmaW5hbHMgMjAyNA%3D%3D',
  'raw_content': '# GABRIEL VS JUSTIN - Pokémon TCG Se

: 